In [36]:
from operator import attrgetter
from collections import Counter
import numpy as np

In [37]:
class Crop:
    def __init__(self, nutrients, indicator, max_val):

        self._nutrients = nutrients
        self._contributions = []

        self._indicator = indicator
        self._max = max_val
        self._used = 0

        self.score = 0

    def getIndicator(self):
        return self._indicator

    def getNutrients(self):
        return self._nutrients

    def getMax(self):
        return self._max

    def getUsed(self):
        return self._used

    def getVariance(self):
        return np.std(self._nutrients)
    
    def isUsedAboveMax(self):
        return (self._used > self._max)
    
    def PopAsRecord(self):
        self._used += 0.01
        return [self._indicator, [n * 0.01 for n in self._nutrients]]
    
    def setUsed(self, increment):
        self._used += increment

    def setContributions(self, maxContributions):
        self._contributions = [(self._nutrients[i] / maxContributions[i]) if maxContributions[i] != 0 else 0.0
            for i in range(len(maxContributions))]
        self.score = np.sum(self._contributions)


In [38]:

class Bucket:
    def __init__(self, maxSkew, maxLevels, percentageMaxTolerance, percentageMinTolerance):

        self._maxSkew = maxSkew
        self._currSkew = 0

        self._maxLevels = maxLevels  
        self._currLevels = [0]*len(self._maxLevels)

        self._maxTolerance = [self._maxLevels[i] + self._maxLevels[i]*percentageMaxTolerance[i] for i in range(len(percentageMaxTolerance))]
        self._minTolerance = [self._maxLevels[i] - self._maxLevels[i]*percentageMinTolerance[i] for i in range(len(percentageMinTolerance))]

        self._maxContributions = [0]*len(self._maxLevels)
        self._records = []
        self._crops = []
        self._cropsExhausted = []

    def addCrop(self, crop):

        self._crops.append(crop)

        for i in range(len(self._maxContributions)):
            self._maxContributions[i] = max(self._crops, key=lambda x: x.getNutrients()[i]).getNutrients()[i]
        
        for crop in self._crops:
            crop.setContributions(self._maxContributions)

        self._crops.sort(key=lambda x: x.score, reverse=True)
    
    def removeCrop(self, crop):
        self._crops.remove(crop)
        self._crops.sort(key=lambda x: x.score, reverse=True)

    def getCrops(self):
        return self._crops
    
    def getCropsByIndicator(self, indicator):
        for crop in self._crops:
            if crop.getIndicator() == indicator:
                return crop

    def flagCropAsExhausted(self, cropIndicator):
        self._cropsExhausted.append(cropIndicator)

    def addToRecords(self, record):
        self._records.append(record)
        for i in range(len(self._currLevels)):
            self._currLevels[i] += record[1][i]
        self._currSkew = np.std([self._currLevels[i]/self._maxLevels[i] for i in range(len(self._maxLevels))])
    
    def animateGraph(self):
        for record in self._records:
            pass

    def isCurrSkewUnderMax(self):
        return (self._currSkew < self._maxSkew)
    
    def isAnyLevelCrossedMax(self):
        for i in range(len(self._maxLevels)):
            if self._currLevels[i] > self._maxTolerance[i]:
                return True
        return False

    def isAllLevelsReachedMin(self):
        for i in range(len(self._maxLevels)):
            if self._currLevels[i] < self._minTolerance[i]:
                return False
        return True

    def getRecords(self):
        return self._records
    
    def getCurrLevels(self):
        return self._currLevels
    
    def getCurrSkew(self):
        return self._currSkew

    def getBestCropByScore(self):
        if not self._crops:
            return None
        return max(self._crops, key=attrgetter('score'))
    
    def getBestCropByStd(self, curr_crop):
        if not self._crops:
            return None
            
        if curr_crop.getIndicator() not in self._cropsExhausted:
            nextBestCrop = curr_crop
        else:
            nextBestCrop = self._crops[0]

        bestDev = self._currSkew

        for crop in self._crops:
            if crop.getIndicator() not in self._cropsExhausted:
                newLevels = [self._currLevels[i] + 0.01 * crop.getNutrients()[i] for i in range(len(self._currLevels))]
                newFills = [newLevels[i]/self._maxLevels[i] for i in range(len(self._maxLevels))]
                newDev = np.std(newFills)

                if newDev < bestDev:
                    nextBestCrop = crop
                    bestDev = newDev
                    break;

        return nextBestCrop

        

In [39]:
#The nutrient order here is Carbohydrates(g/100g-Crop), Protein(g/100g-Crop), Fat(g/100g-Crop), Dietary Fiber (g/100g-Crop), and Energy (kcal/100 g-Crop)

#To temporarily prevent a crop from being added to the bucket, comment out that element within the all_crops tuple. 

all_crops = [
   ('Mushroom',     Crop([6.94, 2.9, 0.19,	2.8, 41 ], 'M',  999)),
   ('Peanut',       Crop([26.5, 23.2, 43.3, 8.0,  588], 'P', 999)),
   ('Rice',         Crop([80.3, 7.04, 1.03, 0.1,  359], 'R', 999)),
   ('DryBean',      Crop([39.7, 21.3, 1.16, 4.0,  52.2], 'D', 999)),
   ('Soybean',      Crop([11.0, 13.0, 6.8,  4.2,  147],  'S', 999)),
   ('Wheat',        Crop([42.5, 7.49, 1.27, 1.1,  198],  'W', 999)),
   ('Pea',          Crop([14.4, 5.42, 0.4,  5.7,  339],  'Pe',999)),
   ('SweetPotato',  Crop([17.3, 1.58, 0.38, 4.44, 79 ], 'Sw', 999)),
   ('Strawberry',   Crop([7.96, 0.64, 0.22, 0.0,  36 ],  'St',999)),
   ('SnapBean',     Crop([7.41,1.97, 0.28, 3.0,   40],   'Sn',999)),
   ('RedBeet',      Crop([9.56,1.61, 0.17, 2.8,   43],   'Re',999)),
   ('WhitePotato',  Crop([15.7,1.68, 0.1,  2.4,   69],   'Wh',999)),
   ('Pepper',       Crop([6.65,0.9,  0.13, 1.2,   31],   'Pp',999)),
   ('Carrot',       Crop([10.3,0.94,0.35, 3.1,   48],    'C', 999)),
   ('Tomato',       Crop([3.84,0.7,  0.42, 1.0,   22],   'T', 999)),
   ('Radish',       Crop([3.4, 0.68, 0.1,  1.6,   16],   'Ra',999)),
   ('Chard',        Crop([3.74,1.8,  0.2,  1.6,   19],   'Ch',999)),
   ('Onion',        Crop([9.34,1.1,  0.1,  1.7,   40],   'O', 999)),
   ('GreenOnion',   Crop([5.74,0.97,1.8,  1.8,   27],    'G', 999)),
   ('Celery',       Crop([2.97,0.69,0.17,1.6,   14],     'Ce',999)),
   ('Spinach',      Crop([3.63,2.86,0.39,2.2,   23],     'Sp',999)),
   ('Cabbage',      Crop([5.8, 1.28,0.1,  2.5,   25],     'Ca',999)),
   ('Lettuce',      Crop([4.06,0.98,0.07,0.0,   21],     'L', 999))
]

bucket = Bucket(0.001,[300,	50,	65,	25,2000],[0.99,0.99,0.99,0.99,0.99],[0.01,0.01,0.01,0.01,0.01])

for crop_name, Crop in all_crops:
    bucket.addCrop(Crop)

In [40]:
for crop in bucket.getCrops():
    print(crop.getIndicator(),crop.score)

P 4.3300124533001245
D 2.0280648117610864
R 1.9502800224175039
Pe 1.711216698982029
S 1.6293750088636911
W 1.355675030797981
Sw 0.9816752634511311
Wh 0.6875870126562975
C 0.646002026600823
M 0.6355417847828463
Sn 0.62668647061267
Re 0.6155054496130442
Sp 0.49160391618599025
Ca 0.48472803014028687
O 0.44656429597311115
G 0.42578109368842065
Ch 0.361093412176715
Pp 0.32733094718061667
Ra 0.30117191842691077
Ce 0.29446330148742217
T 0.2501078213108225
St 0.19301979709252876
L 0.13013269170575414


In [41]:
def forceCrops(dictCrops):
    for key,value in dictCrops.items():
        crop = bucket.getCropsByIndicator(key)
        for i in range(value):
            cropRecord = crop.PopAsRecord()
            bucket.addToRecords(cropRecord)


In [42]:
crop = bucket.getBestCropByScore()
while(crop is not None and not bucket.isAnyLevelCrossedMax() and not bucket.isAllLevelsReachedMin()):
    cropRecord = crop.PopAsRecord()
    bucket.addToRecords(cropRecord)

    print('\n Crop: ',cropRecord, 'Used:', crop.getUsed(),'/',crop.getMax())
    print('Levels', bucket.getCurrLevels())
    print('Deviation',bucket.getCurrSkew())
    if crop.isUsedAboveMax():
        bucket.flagCropAsExhausted(crop.getIndicator())
        bucket.removeCrop(crop)
    crop = bucket.getBestCropByScore() if bucket.isCurrSkewUnderMax() else bucket.getBestCropByStd(crop)


 Crop:  ['P', [0.265, 0.23199999999999998, 0.433, 0.08, 5.88]] Used: 0.01 / 999
Levels [0.265, 0.23199999999999998, 0.433, 0.08, 5.88]
Deviation 0.0019188218881718243

 Crop:  ['R', [0.8029999999999999, 0.0704, 0.0103, 0.001, 3.59]] Used: 0.01 / 999
Levels [1.068, 0.3024, 0.44329999999999997, 0.081, 9.469999999999999]
Deviation 0.0013842577216689094

 Crop:  ['Pe', [0.14400000000000002, 0.0542, 0.004, 0.057, 3.39]] Used: 0.01 / 999
Levels [1.2120000000000002, 0.35660000000000003, 0.4473, 0.138, 12.86]
Deviation 0.001123871127401143

 Crop:  ['R', [0.8029999999999999, 0.0704, 0.0103, 0.001, 3.59]] Used: 0.02 / 999
Levels [2.015, 0.42700000000000005, 0.45759999999999995, 0.139, 16.45]
Deviation 0.0010764480686240485

 Crop:  ['Sw', [0.17300000000000001, 0.0158, 0.0038, 0.0444, 0.79]] Used: 0.01 / 999
Levels [2.188, 0.4428, 0.4614, 0.1834, 17.24]
Deviation 0.0007407284055812745

 Crop:  ['P', [0.265, 0.23199999999999998, 0.433, 0.08, 5.88]] Used: 0.02 / 999
Levels [2.4530000000000003, 0.

In [43]:
cropsToUse = [item[0] for item in bucket.getRecords()]

print(Counter(cropsToUse))
print(bucket.getCurrLevels())

#Counter returns the crop with its mass in g

#CurrLevels (below Counter) returns (in order) the total carbohydrate content (g), protein content (g), dietary fiber content (g), fat content (g), and energy content (kcal)


Counter({'St': 2531, 'G': 738, 'P': 104, 'Pe': 43, 'R': 26, 'C': 20, 'Sw': 11, 'Wh': 1})
[302.57880000000733, 52.0245999999988, 64.43680000000214, 25.213400000000224, 1980.029999999883]
